In [11]:
import cv2
import numpy as np
import base64
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [12]:
def calculate_angle(a, b, c):
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(np.degrees(radians))
    return angle if angle <= 180 else 360 - angle

In [13]:
def pushup_correction():
    # VIDEO FEED
    cap = cv2.VideoCapture(0)

    # Push-up counter variables
    pushup_count = 0
    pushup_started = False
    count_correct = 0
    count_not_correct = 0
    message = ""

    # Thresholds for push-up detection
    WRIST_SHOULDER_DISTANCE_THRESHOLD = 0.15
    WRIST_HIP_DISTANCE_THRESHOLD = 0.15
    STRAIGHT_LINE_ANGLE_THRESHOLD = 20
    ELBOW_ANGLE_THRESHOLD = 100
    HIP_SHOULDER_ANGLE_THRESHOLD = 120
    KNEE_HIP_ANGLE_THRESHOLD = 120
    HIP_KNEE_HEEL_ANGLE_THRESHOLD = 10
    SUM_ANGLE_THRESHOLD = 15

    # Variable to keep track of the frame number
    frame_number = 0

    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Get landmarks for relevant body parts
                left_wrist = np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
                left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
                left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
                left_heel = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                                      landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y])
                left_elbow = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y])
                left_knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
                nose = np.array([landmarks[mp_pose.PoseLandmark.NOSE].x,
                                 landmarks[mp_pose.PoseLandmark.NOSE].y])

                # Calculate distances
                wrist_shoulder_distance = np.linalg.norm(left_wrist - left_shoulder)
                wrist_hip_distance = np.linalg.norm(left_wrist - left_hip)

                # Calculate angles
                angle_elbow_deg = calculate_angle(left_shoulder, left_elbow, left_wrist)
                angle_hip_knee_deg = calculate_angle(left_hip, left_knee, left_heel)
                angle_shoulder_hip_deg = calculate_angle(left_shoulder, left_hip, left_heel)
                angle_knee_heel_deg = calculate_angle(left_knee, left_heel, left_hip)
                angle_hip_heel_deg = calculate_angle(left_hip, left_heel, left_knee)

                # Check for push-up
                if None in landmarks:
                    message = "Put your all body in camera"
                else:
                    if wrist_shoulder_distance < WRIST_SHOULDER_DISTANCE_THRESHOLD:
                        if not pushup_started:
                            pushup_started = True
                    elif wrist_hip_distance > WRIST_HIP_DISTANCE_THRESHOLD:
                        if pushup_started:
                            pushup_count += 1
                            # Check if the push-up form is correct and increment count_correct
                            if angle_hip_knee_deg > KNEE_HIP_ANGLE_THRESHOLD:
                                if abs(angle_shoulder_hip_deg - 180) < HIP_SHOULDER_ANGLE_THRESHOLD:
                                    if abs(angle_hip_knee_deg + angle_knee_heel_deg - 180) < SUM_ANGLE_THRESHOLD:
                                        if (nose_point[1] < left_elbow_point[1] or nose_point[1] < right_elbow_point[1]):
                                            count_correct += 1
                                        else:
                                            count_not_correct += 1
                                            pushup_started = False
                                            message = "Low your Soulder"
                                    else:
                                        count_not_correct += 1
                                        pushup_started = False
                                        message = "Straight your Body"
                                else:
                                    count_not_correct += 1
                                    pushup_started = False
                                    message = "Straight your Back"
                            else:
                                count_not_correct += 1
                            pushup_started = False

            except Exception as e:
                print(f"Error: {e}")

            # Render push-up count on the frame
            cv2.putText(image, f'Correction : {message}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, f'Correct: {count_correct}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, f'Not Correct: {count_not_correct}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 255, 0), 2)

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                       mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                                       mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))

            # Display the frame
            cv2.imshow('Video', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Release the video capture object
    cap.release()
    cv2.destroyAllWindows()

In [14]:
def lateral_raise_correction():
    # VIDEO FEED
    cap = cv2.VideoCapture(0)

    # Lateral raise counter variables
    raise_count = 0
    raise_started = False
    count_correct = 0
    count_not_correct = 0
    message = ""

    # Thresholds for lateral raise detection
    WRIST_SHOULDER_DISTANCE_THRESHOLD = 0.15
    WRIST_HIP_DISTANCE_THRESHOLD = 0.15
    ELBOW_ANGLE_THRESHOLD = 160
    SHOULDER_HIP_ANGLE_THRESHOLD = 85
    MIN_FRAMES_BETWEEN_RAISES = 30
    ELBOW_HIP_ANGLE_THRESHOLD = 85

    # Variable to keep track of the frame where the last raise occurred
    last_raise_frame = 0

    # Variable to keep track of the frame number
    frame_number = 0

    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Get landmarks for relevant body parts
                left_wrist = np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
                left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
                left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
                left_elbow = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y])

                # Calculate distances
                wrist_shoulder_distance = np.linalg.norm(left_wrist - left_shoulder)
                wrist_hip_distance = np.linalg.norm(left_wrist - left_hip)

                # Calculate angles
                angle_elbow_deg = calculate_angle(left_shoulder, left_elbow, left_wrist)
                angle_shoulder_hip_deg = calculate_angle(left_shoulder, left_hip, left_wrist)
                angle_elbow_hip_deg = calculate_angle(left_elbow, left_shoulder, left_hip)

                # Increment the frame number
                frame_number += 1

                if wrist_hip_distance > WRIST_HIP_DISTANCE_THRESHOLD:
                    if not raise_started:
                        raise_started = True
                        max_angle = max(angle_elbow_hip_deg,
                                        angle_shoulder_hip_deg)  # Get the maximum angle between elbow and hip
                        if (abs(max_angle - 90) <= 5 or abs(max_angle - 90) > 5):
                            if angle_elbow_deg >= ELBOW_ANGLE_THRESHOLD:
                                if raise_count == 0 or (
                                        raise_count > 0 and frame_number - last_raise_frame >= MIN_FRAMES_BETWEEN_RAISES):
                                    raise_count += 1
                                    count_correct += 1
                                    last_raise_frame = frame_number
                                    raise_started = False
                                else:
                                    raise_started = False
                            else:
                                count_not_correct += 1
                                raise_started = False
                                message = "Make sure your arm is straight"
                        else:
                            count_not_correct += 1
                            raise_started = False
                            message = "Raise your hand"
                else:
                    raise_started = False

            except Exception as e:
                print(f"Error: {e}")

            # Render lateral raise count on the frame
            cv2.putText(image, f'correction: {message}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, f'Correct: {count_correct}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, f'Not Correct: {count_not_correct}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 255, 0), 2)

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                       mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                                       mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))

            # Display the frame
            cv2.imshow('Video', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Release the video capture object
    cap.release()
    cv2.destroyAllWindows()


In [16]:
def main():
    choice = input("\nEnter 1 for push-up correction \n\nEnter 2 for lateral raise correction: ")
    if choice == '1':
        pushup_correction()
    elif choice == '2':
        lateral_raise_correction()
    else:
        print("Invalid choice. Please enter either 1 or 2.")


if __name__ == "__main__":
    main()
